In [2]:
import random as r

from engine import Value
from modules import *
from ops import *
from gpt import *

# addition
should we do modular addition instead??? problem with that is we'd have to use a huge modulo to have a reasonably sized training dataset right?

In [4]:
max_digits = 2

In [7]:
def generate_data(k: int):
    '''
    creates a string representation of an arithmetic addition operation where the digits are in reverse-order
    ex:
    123 + 45 = 168 would be '321+54=861'
    this reversal is meant to make the task easier on teh transformer
    '''
    max_num = int('9' * k)
    num1 = r.randint(0, max_num)
    num2 = r.randint(0, max_num)
    num3 = num1 + num2
    return f'b{str(num1)[::-1]}+{str(num2)[::-1]}={str(num3)[::-1]}e'

In [13]:
dataset = set()
max_possible_dataset_size = int('9' * max_digits)**2
dataset_size = 2
i = 0
while dataset_size < max_possible_dataset_size:
    i += 1
    dataset_size = 2 ** i
dataset_size = 2 ** (i - 1)
assert dataset_size < max_possible_dataset_size
while len(dataset) < dataset_size:
    dataset.add(generate_data(max_digits))
#print(dataset)

In [15]:
split_size = int(0.95 * len(dataset))
dataset = list(dataset)
train_dataset = dataset[:split_size]
val_dataset = dataset[split_size:]
print(len(train_dataset))
print(len(val_dataset))

7782
410


In [17]:
tokenizer = {
    '0':0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 
    '+':10, '=':11, 
    'b':12, # beginning of sequence token
    'e':13, # end of sequence token
    'p':14 # padding token
}
v = len(tokenizer)
max_data_len = 4 + max_digits * 3 + 1
def tokenize(equation):
    out = []
    for c in equation:
        out.append(tokenizer[c])
    while len(out) < max_data_len:
        out.append(tokenizer['p'])
    return out[:-1], out[1:]
print(tokenize(train_dataset[0]))

([12, 3, 4, 10, 7, 9, 11, 0, 4, 1], [3, 4, 10, 7, 9, 11, 0, 4, 1, 13])


In [42]:
batch_size = 16
train_iterations = split_size // batch_size
print(train_iterations)
val_iterations = (dataset_size - split_size) // batch_size
print(val_iterations)
val_frequency = train_iterations // val_iterations
print(val_frequency)

config = Config()
config.max_seq_len = max_data_len - 1
config.vocab_len = v
model = GPT(config)


486
25
19


In [46]:
for i in range(val_frequency + 1):#train_iterations):
        
    if i % val_frequency == 0:
        val_batch_data = val_dataset[i % val_frequency:(i % val_frequency) + batch_size]
        val_input_toks, val_target_toks = [], []
        for entry in val_batch_data:
            val_input_toks_seq, val_target_toks_seq = tokenize(entry)
            val_input_toks.append(val_input_toks_seq)
            val_target_toks.append(val_target_toks_seq)
            
        probabilities, loss = gpt(val_input_toks, val_target_toks)
        print(f'step {i} loss: {loss}')
        
    train_batch_data = train_dataset[i:i + batch_size]
    train_input_toks, train_target_toks = [], []
    for entry in train_batch_data:
        train_input_toks_seq, train_target_toks_seq = tokenize(entry)
        train_input_toks.append(train_input_toks_seq)
        train_target_toks.append(train_target_toks_seq)

    probabilities, loss = model(train_input_toks, train_target_toks)

    ## backward pass
    #set params to 0
    for p in model.parameters():
        p.grad = 0.0
    # clac gradients
    loss.backward()
    # performing a step of SGD
    for p in model.parameters():
        p.data += -eta * p.grad

step 0 loss: Value(data=433.288, grad=0.000)
step 19 loss: Value(data=433.288, grad=0.000)
